In [1]:
!pip install biopython

In [3]:
from Bio import SeqIO
import io
import os
from collections import defaultdict

In [5]:
from Bio import SeqIO
import numpy as np
import random

input_fasta = "/Users/rohannrahulshah/AFML Project/uniprotkb_family_protein_kinase_AND_rev_2025_09_05.fasta_unreviewed (1)"
output_fasta = "kinases_subset_100k.fasta"

# Collect sequences
records = [(record.id, str(record.seq)) for record in SeqIO.parse(input_fasta, "fasta")]

# Filter out non-standard amino acids
valid_aas = set("ACDEFGHIKLMNPQRSTVWY")
records = [(id, seq) for id, seq in records if set(seq) <= valid_aas]

# Deduplicate
records = list({seq: id for id, seq in records}.items())  # [(seq, id)]
records = [(id, seq) for seq, id in records]              # back to (id, seq)

# Compute length distribution
lengths = np.array([len(seq) for _, seq in records])

Q1, Q3 = np.percentile(lengths, [25, 75])
IQR = Q3 - Q1
lower = max(1, Q1 - 1.5 * IQR)   # avoid negative lengths
upper = Q3 + 1.5 * IQR

print(f"Length filtering bounds: {lower:.0f} - {upper:.0f}")

# Apply IQR filter
filtered = [(id, seq) for id, seq in records if lower <= len(seq) <= upper]

print(f"Remaining after IQR filter: {len(filtered)} sequences")

# Sample 100k (if enough left)
random.seed(42)
subset = random.sample(filtered, 100000)

# Save to FASTA
with open(output_fasta, "w") as f:
    for id, seq in subset:
        f.write(f">{id}\n{seq}\n")

print(f"Saved 100k sequences to {output_fasta}")


Length filtering bounds: 1 - 1408
Remaining after IQR filter: 1049384 sequences
Saved 100k sequences to kinases_subset_100k.fasta


In [12]:
import os
import random
import subprocess
from Bio import SeqIO
import numpy as np
from collections import defaultdict

# =====================
# Config
# =====================
input_fasta   = "/Users/rohannrahulshah/AFML Project/uniprotkb_family_protein_kinase_AND_rev_2025_09_05.fasta_unreviewed (1)"
work_dir      = "mmseqs_work"
sample_size   = 100000
seed          = 42
split_ratio   = (0.8, 0.1, 0.1)
identity_thr  = 0.4  # 40% identity

os.makedirs(work_dir, exist_ok=True)

# =====================
# Step 1: Load & filter canonical AAs
# =====================
valid_aas = set("ACDEFGHIKLMNPQRSTVWY")
records = []
for record in SeqIO.parse(input_fasta, "fasta"):
    seq = str(record.seq).upper()
    if set(seq) <= valid_aas:
        records.append((record.id, seq))
print(f"Loaded {len(records)} canonical sequences.")

# =====================
# Step 2: Deduplicate exact sequences
# =====================
unique = {seq: rid for rid, seq in records}
deduped = [(rid, seq) for seq, rid in unique.items()]
print(f"After deduplication: {len(deduped)} sequences.")

# =====================
# Step 3: IQR-based length filter
# =====================
lengths = np.array([len(seq) for _, seq in deduped])
Q1, Q3 = np.percentile(lengths, [25, 75])
IQR = Q3 - Q1
lower = max(1, int(Q1 - 1.5 * IQR))
upper = int(Q3 + 1.5 * IQR)

filtered = [(rid, seq) for rid, seq in deduped if lower <= len(seq) <= upper]
print(f"IQR bounds: {lower} - {upper}")
print(f"After IQR filtering: {len(filtered)} sequences.")

# =====================
# Step 4: Random sample
# =====================
if len(filtered) < sample_size:
    raise ValueError(f"Only {len(filtered)} sequences left, cannot sample {sample_size}.")
random.seed(seed)
subset = random.sample(filtered, sample_size)
print(f"Randomly sampled {sample_size} sequences.")

subset_fasta = os.path.join(work_dir, "kinases_100k.fasta")
with open(subset_fasta, "w") as f:
    for rid, seq in subset:
        f.write(f">{rid}\n{seq}\n")

# =====================
# Step 5: MMseqs2 clustering
# =====================
db_file = os.path.join(work_dir, "kinasesDB")
clu_file = os.path.join(work_dir, "kinasesDB_clu")
tsv_file = os.path.join(work_dir, "kinasesDB_clu.tsv")
tmp_dir  = os.path.join(work_dir, "tmp")
os.makedirs(tmp_dir, exist_ok=True)

print("Running MMseqs2 clustering...")
subprocess.run(["mmseqs", "createdb", subset_fasta, db_file], check=True)
subprocess.run([
    "mmseqs", "cluster", db_file, clu_file, tmp_dir,
    "--min-seq-id", str(identity_thr), "-c", "0.8"
], check=True)
subprocess.run(["mmseqs", "createtsv", db_file, db_file, clu_file, tsv_file], check=True)
print(f"Clustering done. Results in {tsv_file}")

# =====================
# Step 6: Parse clusters & remap IDs
# =====================
clusters = defaultdict(list)
with open(tsv_file) as f:
    for line in f:
        rep, mem = line.strip().split("\t")
        clusters[rep].append(mem)

# Map MMseqs2 short IDs back to original FASTA headers
id_map = {}
for rec in SeqIO.parse(subset_fasta, "fasta"):
    full_id = rec.id
    short_id = full_id.split("|")[1] if "|" in full_id else full_id
    id_map[short_id] = full_id

clusters_remapped = defaultdict(list)
for rep, members in clusters.items():
    rep_full = id_map.get(rep, rep)
    members_full = [id_map.get(m, m) for m in members]
    clusters_remapped[rep_full] = members_full

cluster_ids = list(clusters_remapped.keys())
random.seed(seed)
random.shuffle(cluster_ids)

# Split by clusters
n = len(cluster_ids)
n_train = int(split_ratio[0]*n)
n_val   = int(split_ratio[1]*n)

train_c = cluster_ids[:n_train]
val_c   = cluster_ids[n_train:n_train+n_val]
test_c  = cluster_ids[n_train+n_val:]

train_ids = [s for c in train_c for s in clusters_remapped[c]]
val_ids   = [s for c in val_c for s in clusters_remapped[c]]
test_ids  = [s for c in test_c for s in clusters_remapped[c]]

print(f"Clusters: {n}")
print(f"Split sizes: train={len(train_ids)}, val={len(val_ids)}, test={len(test_ids)}")

# =====================
# Step 7: Save FASTA splits
# =====================
all_records = {rec.id: rec for rec in SeqIO.parse(subset_fasta, "fasta")}

def write_fasta(ids, filename):
    saved = 0
    with open(filename, "w") as out:
        for i in ids:
            if i in all_records:
                out.write(f">{i}\n{all_records[i].seq}\n")
                saved += 1
    print(f"Saved {saved} sequences to {filename}")

write_fasta(train_ids, "kinases_cluster_train.fasta")
write_fasta(val_ids,   "kinases_cluster_val.fasta")
write_fasta(test_ids,  "kinases_cluster_test.fasta")

print("✅ Done. Files saved:")
print(" - kinases_cluster_train.fasta")
print(" - kinases_cluster_val.fasta")
print(" - kinases_cluster_test.fasta")


Loaded 1184764 canonical sequences.
After deduplication: 1100707 sequences.
IQR bounds: 1 - 1408
After IQR filtering: 1049384 sequences.
Randomly sampled 100000 sequences.
Running MMseqs2 clustering...
createdb mmseqs_work/kinases_100k.fasta mmseqs_work/kinasesDB 

MMseqs Version:                    	18-8cc5c
Database type                      	0
Shuffle input database             	true
Createdb mode                      	0
Write lookup file                  	1
Offset of numeric ids              	0
Threads                            	8
Compressed                         	0
Mask residues                      	0
Mask residues probability          	0.9
Mask lower case residues           	0
Mask lower letter repeating N times	0
Use GPU                            	0
Verbosity                          	3

Converting sequences
[=========
Time for merging to kinasesDB_h: 0h 0m 0s 35ms
Time for merging to kinasesDB: 0h 0m 0s 59ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 464ms
clus